In [1]:
from tohu import Integer, HashDigest, FakerGenerator
from tohu.custom_generator_NEW import CustomGeneratorNEW
from tohu.utils import print_generated_sequence

#### Custom generator without `__init__` method

The simplest way to use `CustomGenerator` is with only class attributes (without `__init__` method).

In [2]:
class QuuxGenerator(CustomGeneratorNEW):
    aa = Integer(100, 200)
    bb = HashDigest(length=6)
    cc = FakerGenerator(method="name")

In [3]:
g = QuuxGenerator()

2020-04-18 22:24:55 INFO  [DDD] Inside CustomGeneratorNEW.__init__()


In [4]:
print_generated_sequence(g, num=5, seed=11111, sep="\n")

Generated sequence:

Quux(aa=163, bb='7551AA', cc='Michelle Miller')
Quux(aa=171, bb='54596E', cc='Eddie Davis')
Quux(aa=142, bb='2A16D0', cc='Kathleen Lucas')
Quux(aa=140, bb='FDCDD3', cc='Jason Rodriguez')
Quux(aa=121, bb='BDE283', cc='Andrew Pitts')


Note that while the _instance_ `g` contains attributes with the same names `"aa"` and `"bb"` as the parent class `QuuxGenerator`, internally the instance attributes are actually spawned versions of the class attributes (in particular, they are not identical):

In [5]:
print(QuuxGenerator.aa)
print(g.aa)
assert QuuxGenerator.aa is not g.aa

print(QuuxGenerator.bb)
print(g.bb)
assert QuuxGenerator.bb is not g.bb

print(QuuxGenerator.cc)
print(g.cc)
assert QuuxGenerator.cc is not g.cc

<Integer (id=dab91d)>
<Integer (id=924e40)>
<HashDigest (id=0b034c)>
<HashDigest (id=7d5d55)>
<FakerGenerator (id=c7c0db)>
<FakerGenerator (id=d3dce0)>


#### Custom generator with `__init__` method

Tohu generators that are defined as instance attributes within the `__init__` method are also picked up.

In [6]:
class QuuxGenerator(CustomGeneratorNEW):
    aa = Integer(100, 200)
    bb = HashDigest(length=6)
    
    def __init__(self, method):
        self.cc = FakerGenerator(method=method)

In [7]:
g = QuuxGenerator(method="first_name")

2020-04-18 22:24:55 INFO  [DDD] Inside CustomGeneratorNEW.__init__()


In [8]:
print_generated_sequence(g, num=5, seed=11111, sep="\n")

Generated sequence:

Quux(aa=163, bb='7551AA', cc='Mary')
Quux(aa=171, bb='54596E', cc='Jennifer')
Quux(aa=142, bb='2A16D0', cc='Christopher')
Quux(aa=140, bb='FDCDD3', cc='Joseph')
Quux(aa=121, bb='BDE283', cc='Kelly')
